In [36]:
import pyperclip as clip

In [37]:
races = ['BL', 'WH', 'AM', 'AS', 'HI', 'HP', 'TR']
genders = ['M', 'F', 'both_genders']
suspensions = ['MULTOOS', 'SINGOOS']
disciplinary_metrics = ['REF', 'ARR']
academic_metrics = ['GTENR', 'APENR']
disability = ['WODIS', 'WDIS']

### District rate calculations

In [38]:
dist_query_segments = 'SELECT "LEAID"'

### Sum multiple and single suspensions to get total suspensions.

In [39]:
disab=''
for dis_status in disability:
    for race in races:
        if dis_status == 'WDIS':
            disab = 'IDEA_'
        for gender in genders:
            if gender == 'both_genders':
                district_query = f'\n, ("SCH_DISC{dis_status}_SINGOOS_{disab}{race}_M" + "SCH_DISC{dis_status}_SINGOOS_{disab}{race}_F" + "SCH_DISC{dis_status}_MULTOOS_{disab}{race}_F" + "SCH_DISC{dis_status}_MULTOOS_{disab}{race}_M") / (NULLIF("SCH_{disab[:4]}ENR_{race}_F", 0) + NULLIF("SCH_{disab[:4]}ENR_{race}_M", 0)) as {race}_{gender}_{dis_status}_suspension_rate'

            else:
                district_query = f'\n, ("SCH_DISC{dis_status}_MULTOOS_{disab}{race}_{gender}" + "SCH_DISC{dis_status}_SINGOOS_{disab}{race}_{gender}") / NULLIF("SCH_{disab[:4]}ENR_{race}_{gender}", 0) as {race}_{gender}_{dis_status}_suspension_rate'

            dist_query_segments += district_query

In [40]:
for metric in disciplinary_metrics:
    for dis_status in disability:
        if dis_status == 'WDIS':
            metric += '_IDEA'
            disab = 'IDEA'
        if dis_status == 'WODIS':
            disab=''
        for race in races:
            for gender in genders:
                if gender == 'both_genders':
                    district_query = f'\n, ("SCH_DISC{dis_status}_{metric}_{race}_F" + "SCH_DISC{dis_status}_{metric}_{race}_M") / (NULLIF("SCH_{disab}ENR_{race}_F", 0) + NULLIF("SCH_{disab}ENR_{race}_M", 0)) as {race}_{gender}_{dis_status}_{metric}_rate'

                else:
                    district_query = f'\n, ("SCH_DISC{dis_status}_{metric}_{race}_{gender}" / NULLIF("SCH_{disab}ENR_{race}_{gender}", 0)) as {race}_{gender}_{dis_status}_{metric}_rate'

                dist_query_segments += district_query

In [41]:
print(dist_query_segments)

SELECT "LEAID"
, ("SCH_DISCWODIS_MULTOOS_BL_M" + "SCH_DISCWODIS_SINGOOS_BL_M") / NULLIF("SCH_ENR_BL_M", 0) as BL_M_WODIS_suspension_rate
, ("SCH_DISCWODIS_MULTOOS_BL_F" + "SCH_DISCWODIS_SINGOOS_BL_F") / NULLIF("SCH_ENR_BL_F", 0) as BL_F_WODIS_suspension_rate
, ("SCH_DISCWODIS_SINGOOS_BL_M" + "SCH_DISCWODIS_SINGOOS_BL_F" + "SCH_DISCWODIS_MULTOOS_BL_F" + "SCH_DISCWODIS_MULTOOS_BL_M") / (NULLIF("SCH_ENR_BL_F", 0) + NULLIF("SCH_ENR_BL_M", 0)) as BL_both_genders_WODIS_suspension_rate
, ("SCH_DISCWODIS_MULTOOS_WH_M" + "SCH_DISCWODIS_SINGOOS_WH_M") / NULLIF("SCH_ENR_WH_M", 0) as WH_M_WODIS_suspension_rate
, ("SCH_DISCWODIS_MULTOOS_WH_F" + "SCH_DISCWODIS_SINGOOS_WH_F") / NULLIF("SCH_ENR_WH_F", 0) as WH_F_WODIS_suspension_rate
, ("SCH_DISCWODIS_SINGOOS_WH_M" + "SCH_DISCWODIS_SINGOOS_WH_F" + "SCH_DISCWODIS_MULTOOS_WH_F" + "SCH_DISCWODIS_MULTOOS_WH_M") / (NULLIF("SCH_ENR_WH_F", 0) + NULLIF("SCH_ENR_WH_M", 0)) as WH_both_genders_WODIS_suspension_rate
, ("SCH_DISCWODIS_MULTOOS_AM_M" + "SCH_DISCWODI

In [42]:
for metric in academic_metrics:
    for race in races:
        for gender in genders:
            if gender == 'both_genders':
                district_query = f'\n, ("SCH_{metric}_{race}_F" + "SCH_{metric}_{race}_M") / (NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0)) as {race}_{gender}_{metric}_rate'

            else:
                district_query = f'\n, ("SCH_{metric}_{race}_{gender}" / NULLIF("SCH_ENR_{race}_{gender}", 0)) as {race}_{gender}_{metric}_rate'

            dist_query_segments += district_query

In [43]:
dist_query_segments += ' from districts;'
# print(dist_query_segments)

In [44]:
clip.copy(dist_query_segments)

### National Rate calculations

Calculate national rate for:
1. White student groups for each metric
2. All student groups for each metric

In [11]:
national_query = 'Select 1'

In [12]:
race = 'WH'

disab=''
for dis_status in disability:
    if dis_status == 'WDIS':
        disab = 'IDEA_'
    for gender in genders:
        if gender == 'both_genders':
            query_segment = f'\n, SUM("SCH_DISC{dis_status}_SINGOOS_{disab}{race}_M" + "SCH_DISC{dis_status}_SINGOOS_{disab}{race}_F" + "SCH_DISC{dis_status}_MULTOOS_{disab}{race}_F" + "SCH_DISC{dis_status}_MULTOOS_{disab}{race}_M") / SUM(NULLIF("SCH_{disab[:4]}ENR_{race}_F", 0) + NULLIF("SCH_{disab[:4]}ENR_{race}_M", 0)) as ntl_{race}_{gender}_{dis_status}_suspension_rate'

        else:
            query_segment = f'\n, SUM("SCH_DISC{dis_status}_MULTOOS_{disab}{race}_{gender}" + "SCH_DISC{dis_status}_SINGOOS_{disab}{race}_{gender}") / SUM(NULLIF("SCH_{disab[:4]}ENR_{race}_{gender}", 0)) as ntl_{race}_{gender}_{dis_status}_suspension_rate'

        national_query += query_segment

disab=''
for metric in disciplinary_metrics:
    for dis_status in disability:
        if dis_status == 'WDIS':
            metric += '_IDEA'
            disab='IDEA'
        for gender in genders:
            if gender == 'both_genders':
                query_segment = f'\n, SUM("SCH_DISC{dis_status}_{metric}_{race}_F" + "SCH_DISC{dis_status}_{metric}_{race}_M") / SUM((NULLIF("SCH_{disab}ENR_{race}_F", 0) + NULLIF("SCH_{disab}ENR_{race}_M", 0))) as ntl_{race}_{gender}_{dis_status}_{metric}_rate'

            else:
                query_segment = f'\n, SUM("SCH_DISC{dis_status}_{metric}_{race}_{gender}") / SUM(NULLIF("SCH_{disab}ENR_{race}_{gender}", 0)) as ntl_{race}_{gender}_{dis_status}_{metric}_rate'

            national_query += query_segment
            
for metric in academic_metrics:
    for gender in genders:
        if gender == 'both_genders':
            query_segment = f'\n, SUM("SCH_{metric}_{race}_F" + "SCH_{metric}_{race}_M") / SUM(NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0)) as ntl_{race}_{gender}_{metric}_rate'

        else:
            query_segment = f'\n, SUM("SCH_{metric}_{race}_{gender}") / SUM(NULLIF("SCH_ENR_{race}_{gender}", 0)) as ntl_{race}_{gender}_{metric}_rate'

        national_query += query_segment

In [13]:
national_query += ' from districts;'

In [14]:
clip.copy(national_query)

Aggregates across all races

In [15]:
all_races_qry = 'SELECT 1'

for dis_status in disability:
    if dis_status == 'WDIS':
        disab = 'IDEA_'
    if dis_status == 'WODIS':
        disab=''
    for gender in genders:
        if gender == 'both_genders':
            query_segment = f'\n, SUM("TOT_DISC{dis_status}_MULTOOS_{disab}F" +"TOT_DISC{dis_status}_SINGOOS_{disab}F" + "TOT_DISC{dis_status}_MULTOOS_{disab}M" + "TOT_DISC{dis_status}_SINGOOS_{disab}F") / SUM((NULLIF("TOT_{disab[:4]}ENR_F", 0) + NULLIF("TOT_{disab[:4]}ENR_M", 0))) as ntl_all_races_{gender}_{dis_status}_suspension_rate'

        else:
            query_segment = f'\n, SUM("TOT_DISC{dis_status}_MULTOOS_{disab}{gender}" + "TOT_DISC{dis_status}_SINGOOS_{disab}{gender}") / SUM(NULLIF("TOT_{disab[:4]}ENR_{gender}", 0)) as ntl_all_races_{gender}_{dis_status}_suspension_rate'

        all_races_qry += query_segment

for metric in disciplinary_metrics:
    for dis_status in disability:
        if dis_status == 'WDIS':
            metric += '_IDEA'
            disab = 'IDEA'
        if dis_status == 'WODIS':
            disab = ''
        for gender in genders:
            if gender == 'both_genders':
                query_segment = f'\n, SUM("TOT_DISC{dis_status}_{metric}_F" + "TOT_DISC{dis_status}_{metric}_M") / SUM((NULLIF("TOT_{disab}ENR_F", 0) + NULLIF("TOT_{disab}ENR_M", 0))) as ntl_all_races_{gender}_{dis_status}_{metric}_rate'

            else:
                query_segment = f'\n, SUM("TOT_DISC{dis_status}_{metric}_{gender}") / SUM(NULLIF("TOT_{disab}ENR_{gender}", 0)) as ntl_all_races_{gender}_{dis_status}_{metric}_rate'

            all_races_qry += query_segment
            

for metric in academic_metrics:
    for gender in genders:
        if gender == 'both_genders':
            query_segment = f'\n, SUM("TOT_{metric}_F" + "TOT_{metric}_M") / SUM(NULLIF("TOT_ENR_F", 0) + NULLIF("TOT_ENR_M", 0)) as ntl_all_races_{gender}_{metric}_rate'

        else:
            query_segment = f'\n, SUM("TOT_{metric}_{gender}") / SUM(NULLIF("TOT_ENR_{gender}", 0)) as ntl_all_races_{gender}_{metric}_rate'

        all_races_qry += query_segment
        

In [16]:
clip.copy(all_races_qry)

### Generate risk ratio queries

In [68]:
ratio_segments = 'SELECT 1 '

for metric in disciplinary_metrics + ['suspension']:
    for dis_status in disability:
        if dis_status == 'WDIS' and metric != 'suspension':
            metric += '_IDEA'
        for race in races:
            for gender in genders:
                all_race_ratio_query = f'\n, ("{race}_{gender}_{dis_status}_{metric}_rate" / "ntl_all_races_{gender}_{dis_status}_{metric}_rate") as {race}_{gender}_{dis_status}_{metric}_risk_ratio_all_races'
                white_student_ratio_query = f'\n, ("{race}_{gender}_{dis_status}_{metric}_rate" / "ntl_WH_{gender}_{dis_status}_{metric}_rate") as {race}_{gender}_{dis_status}_{metric}_risk_ratio_white'
                ratio_segments += all_race_ratio_query.lower()
                ratio_segments += white_student_ratio_query.lower()

# ratio_segments += ' from (select * from district_rates cross join national_rates) full_table;'
print(ratio_segments)

SELECT 1 
, ("bl_m_wodis_ref_rate" / "ntl_all_races_m_wodis_ref_rate") as bl_m_wodis_ref_risk_ratio_all_races
, ("bl_m_wodis_ref_rate" / "ntl_wh_m_wodis_ref_rate") as bl_m_wodis_ref_risk_ratio_white
, ("bl_f_wodis_ref_rate" / "ntl_all_races_f_wodis_ref_rate") as bl_f_wodis_ref_risk_ratio_all_races
, ("bl_f_wodis_ref_rate" / "ntl_wh_f_wodis_ref_rate") as bl_f_wodis_ref_risk_ratio_white
, ("bl_both_genders_wodis_ref_rate" / "ntl_all_races_both_genders_wodis_ref_rate") as bl_both_genders_wodis_ref_risk_ratio_all_races
, ("bl_both_genders_wodis_ref_rate" / "ntl_wh_both_genders_wodis_ref_rate") as bl_both_genders_wodis_ref_risk_ratio_white
, ("wh_m_wodis_ref_rate" / "ntl_all_races_m_wodis_ref_rate") as wh_m_wodis_ref_risk_ratio_all_races
, ("wh_m_wodis_ref_rate" / "ntl_wh_m_wodis_ref_rate") as wh_m_wodis_ref_risk_ratio_white
, ("wh_f_wodis_ref_rate" / "ntl_all_races_f_wodis_ref_rate") as wh_f_wodis_ref_risk_ratio_all_races
, ("wh_f_wodis_ref_rate" / "ntl_wh_f_wodis_ref_rate") as wh_f_wodis_

In [69]:
for metric in academic_metrics:
    for race in races:
        for gender in genders:
            all_race_ratio_query = f'\n, ("ntl_all_races_{gender}_{metric}_rate" / NULLIF("{race}_{gender}_{metric}_rate", 0)) as all_races_ratio_{race}_{gender}_{metric}_risk'
            white_student_ratio_query = f'\n, ("ntl_WH_{gender}_{metric}_rate" / NULLIF("{race}_{gender}_{metric}_rate", 0)) as white_ratio_{race}_{gender}_{metric}_risk'
            ratio_segments += all_race_ratio_query.lower()
            ratio_segments += white_student_ratio_query.lower()


In [70]:
print(ratio_segments)

SELECT 1 
, ("bl_m_wodis_ref_rate" / "ntl_all_races_m_wodis_ref_rate") as bl_m_wodis_ref_risk_ratio_all_races
, ("bl_m_wodis_ref_rate" / "ntl_wh_m_wodis_ref_rate") as bl_m_wodis_ref_risk_ratio_white
, ("bl_f_wodis_ref_rate" / "ntl_all_races_f_wodis_ref_rate") as bl_f_wodis_ref_risk_ratio_all_races
, ("bl_f_wodis_ref_rate" / "ntl_wh_f_wodis_ref_rate") as bl_f_wodis_ref_risk_ratio_white
, ("bl_both_genders_wodis_ref_rate" / "ntl_all_races_both_genders_wodis_ref_rate") as bl_both_genders_wodis_ref_risk_ratio_all_races
, ("bl_both_genders_wodis_ref_rate" / "ntl_wh_both_genders_wodis_ref_rate") as bl_both_genders_wodis_ref_risk_ratio_white
, ("wh_m_wodis_ref_rate" / "ntl_all_races_m_wodis_ref_rate") as wh_m_wodis_ref_risk_ratio_all_races
, ("wh_m_wodis_ref_rate" / "ntl_wh_m_wodis_ref_rate") as wh_m_wodis_ref_risk_ratio_white
, ("wh_f_wodis_ref_rate" / "ntl_all_races_f_wodis_ref_rate") as wh_f_wodis_ref_risk_ratio_all_races
, ("wh_f_wodis_ref_rate" / "ntl_wh_f_wodis_ref_rate") as wh_f_wodis_

In [71]:
clip.copy(ratio_segments)